In [1]:
import pandas as pd
from itertools import combinations
import random

# DICTIONARIES: SPLITS, MODALITIES, AND EQUIPMENT PRESELECTION
## FROM calculate_exercises.ipynb

In [2]:
#SIMPLE DICTIONARY THAT MAPS 'SPLITS' (i.e. Push-Pull-Legs) TO MUSCLE GROUPS (i.e. [Chest, Shoulders, Triceps], [Back, Biceps, Trapezius, ...] )
split_dictionary = {
    "Upper-lower"    : [["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Abs","Obliques", "Lower back"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"]],

    "Push-Pull-Legs" : [["Chest","Shoulders", "Triceps"],
                        ["Back", "Biceps","Trapezius", "Abs","Obliques"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves","Lower back"]],

    "PHUL"           : [["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Abs","Obliques", "Lower back"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"]],
    
    "Hybrid PPL+ UL" : [["Chest","Shoulders", "Triceps"],
                        ["Back", "Biceps","Trapezius", "Abs","Obliques"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves","Lower back"],
                        ["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Abs","Obliques", "Lower back"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"]],

    "Body part split": [["Chest"],
                        ["Back","Trapezius"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves", "Lower back"],
                        ["Shoulders","Abs","Obliques"],
                        ["Biceps","Triceps"]],

    "6-day body part split": [["Chest"],
                        ["Back","Trapezius"],
                        ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"],
                        ["Abs","Obliques","Lower back"],
                        ["Shoulders"],
                        ["Biceps","Triceps"]]
}

#DICTIONARY THAT MAPS 'SPLITS' (i.e. Push-Pull-Legs) TO MUSCLE GROUPS (i.e. [Chest, Shoulders, Triceps], [Back, Biceps, Trapezius, ...])
# AND FOCUS DISTRIBUTION RANGES (ACCEPTED WORKOUT DISTRIBUTION PER MUSCLE) 
# AND PROBABILITIES (PROBABILITIES THAT AN EXERCISE FOR THAT MUSCLE WILL BE SELECTED)

split_dictionary_complex = {
    "Upper-lower": {
        "groups": [
            ["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Lower back", "Obliques", "Abs"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves"]
        ],
        "focus_distribution_ranges": [
            [(0.2, 0.35), (0.1, 0.2), (0, 0.15), (0.2, 0.35), (0, 0.15), (0, 0.1), (0, 0.1), (0, 0.1), (0, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15)]
        ],
        "probabilities": [
            [0.25, 0.14, 0.08, 0.25, 0.08, 0.05, 0.03, 0.03, 0.09],
            [0.3, 0.2, 0.3, 0.05, 0.05, 0.1]
        ]
    },

    "Push-Pull-Legs": {
        "groups": [
            ["Chest", "Shoulders", "Triceps"],
            ["Back", "Biceps", "Trapezius", "Abs", "Obliques"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves", "Lower back"]
        ],
        "focus_distribution_ranges": [
            [(0.4, 0.6), (0.2, 0.4), (0.2, 0.4)],
            [(0.4, 0.6), (0.1, 0.3), (0.1, 0.2), (0.1, 0.3), (0.05, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15),(0.05, 0.1)]
        ],
        "probabilities": [
            [0.5, 0.3, 0.2],
            [0.5, 0.2, 0.13, 0.12, 0.05],
            [0.24, 0.2, 0.24, 0.08, 0.08, 0.1, 0.06]
        ]
    },

    "PHUL": {
        "groups": [
            ["Chest", "Shoulders", "Triceps", "Back", "Biceps", "Trapezius", "Lower back", "Obliques", "Abs"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves"]
        ],
        "focus_distribution_ranges": [
            [(0.2, 0.3), (0.1, 0.2), (0.1, 0.15), (0.2, 0.3), (0.1, 0.15), (0.1, 0.15), (0.05, 0.1), (0.05, 0.1), (0.05, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15)]
        ],
        "probabilities": [
            [0.24, 0.16, 0.08, 0.24, 0.08, 0.05, 0.03, 0.03, 0.09],
            [0.3, 0.2, 0.3, 0.05, 0.05, 0.1]
        ]
    },

    "Hybrid PPL+ UL": {
        "groups": [
            ["Chest", "Shoulders", "Triceps"],
            ["Back", "Biceps", "Trapezius", "Abs", "Obliques"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves", "Lower back"],
            ["Chest", "Shoulders", "Triceps", "Back", "Biceps", "Trapezius", "Lower back", "Obliques", "Abs"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves"]
        ],
        "focus_distribution_ranges": [
            [(0.4, 0.6), (0.2, 0.4), (0.2, 0.4)],
            [(0.4, 0.6), (0.1, 0.3), (0.1, 0.2), (0.1, 0.3), (0.05, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15),(0.05, 0.1)],
            [(0.2, 0.3), (0.1, 0.2), (0.1, 0.15), (0.2, 0.3), (0.1, 0.15), (0.1, 0.15), (0.05, 0.1), (0.05, 0.1), (0.05, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15)]
        ],
        "probablities": [
            [0.5, 0.3, 0.2],
            [0.5, 0.2, 0.13, 0.12, 0.05],
            [0.24, 0.2, 0.24, 0.08, 0.08, 0.1, 0.06],
            [0.24, 0.16, 0.08, 0.24, 0.08, 0.05, 0.03, 0.03, 0.09],
            [0.3, 0.2, 0.3, 0.05, 0.05, 0.1]
        ]
    },

    "Body part split": {
        "groups": [
            ["Chest"],
            ["Back", "Trapezius"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves", "Lower back"],
            ["Shoulders", "Abs", "Obliques"],
            ["Biceps", "Triceps"]
        ],
        "focus_distribution_ranges": [
            [(0.8, 1.0)],
            [(0.7, 0.9), (0.3, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15),(0.05, 0.1)],
            [(0.6, 0.8), (0.4, 0.2), (0.2, 0.05)],
            [(0.4, 0.6), (0.4, 0.6)]
        ],
        "probabilities": [
            [1],
            [0.8, 0.2],
            [0.24, 0.2, 0.24, 0.08, 0.08, 0.1, 0.06],
            [0.7, 0.2, 0.1],
            [0.5, 0.5]
        ]
    },

    "6-day body part split": {
        "groups": [
            ["Chest"],
            ["Back", "Trapezius"],
            ["Quads", "Glutes", "Hamstrings", "Abductors", "Adductors", "Calves"],
            ["Abs", "Obliques", "Lower back"],
            ["Shoulders"],
            ["Biceps", "Triceps"]
        ],
        "focus_distribution_ranges": [
            [(0.8, 1.0)],
            [(0.7, 0.9), (0.3, 0.1)],
            [(0.3, 0.4), (0.2, 0.3), (0.2, 0.3), (0.05, 0.1), (0.05, 0.1), (0.1, 0.15)],
            [(0.6, 0.7), (0.2, 0.3), (0.2, 0.3)],
            [(0.8, 1.0)],
            [(0.4, 0.6), (0.4, 0.6)]
        ],
        "probabilities": [
            [1],
            [0.8, 0.2],
            [0.24, 0.2, 0.24, 0.08, 0.08, 0.1, 0.06],
            [0.7, 0.2, 0.1],
            [1],
            [0.5, 0.5]
        ]
    }
}


In [3]:
### FIRST VERSION OF GOAL TO MODALITY ###
#WHERE GOAL IS WHAT THE USER SELECTED ON THE ONBOARDING, AND MODALITIES (i.e. Hypertrophy, Stabiliy, Cardio, etc) IS BASED ON THOSE GOALS

# goal_to_modality = {

#     "Powerlifting"       : ['PL'],

#     "Get stronger"       : ['STR'],

#     "Bodybuilding"       : ['H'],

#     "Build Muscles"  : ['H', 'STR', 'ME'],

#     "Losing Weight"  : ['H', 'STR', 'ME', 'C'],     #should we add C for cardio?

#     "Get Lean"           : ['H', 'ME', 'C'],         #should we add c for cardio?

#     "Aesthetics"         : ['H', 'STR', 'ME'],

#     "Maintain Health"    : ['H','STR','ME','AE','STA','MO','F'], #should we include AE? what about STA, MO, and F?

#     "Increase Endurance" : ['ME', 'AE'],            #should we include AE?

#     "Become Athletic"    : ['AT'],

#     "Injury prevention"  : ['H','ME','STA','B','MO','F'],

#     "Improve balance"    : ['B'],

#     "Improve mobility"   : ['MO']

# }

### GOAL TO MODALITY SIMPLIFIED ###

#Here, I decided to combine 'H', 'STR', and 'ME' into 'H' since they all are the same exercises. We only need to differ the modality
#To choose the number of reps. Might need to defined ranges for STR [5-8], H [9-12] and ME [13-16]. We'll see what approach we take
#since there needs to be variation. 

# goal_to_modality_simple = {

#     "Powerlifting"       : ['PL'],

#     "Get stronger"       : ['H'],

#     "Bodybuilding"       : ['H'],

#     "Build Muscles"  : ['H'],

#     "Losing Weight"  : ['H', 'C'],     #should we add C for cardio?

#     "Get Lean"           : ['H', 'C'],         #should we add c for cardio?

#     "Aesthetics"         : ['H'],

#     "Maintain Health"    : ['H','AE','STA','MO','F'], #should we include AE? what about STA, MO, and F?

#     "Increase Endurance" : ['H', 'AE'],            #should we include AE?

#     "Become Athletic"    : ['AT'],

#     "Injury prevention"  : ['H','STA','B','MO','F'],

#     "Improve balance"    : ['B'],

#     "Improve mobility"   : ['MO']

# }


#IN THE NEXT VERSION, THE FOLLOWING "MODALITIES" ARE IGNORED:

# Powerlifting
# Maintain health
# Become athletic
# Injury prevention
# Improve balance
# Improve mobility

goal_to_modality_further_simplified = {

    "Get stronger"       : ['H'],

    "Bodybuilding"       : ['H'],

    "Build muscles"  : ['H'],

    "Aesthetics"         : ['H'],
    
    "Losing weight"  : ['H', 'C'],     #should we add C for cardio?

    "Get lean"           : ['H', 'C'],         #should we add C for cardio?

    "Increase endurance" : ['H', 'AE']            #should we include AE?

}

#FURTHERMORE 
# When LOSING WEIGHT and/or GET LEAN are selected, we need to allocate TIME and EXERCISES for CARDIO.
# When INCREASE ENDURANCE             is selected, we need to allocate TIME and EXERCISES for AEROBIC ENDURANCE.

In [4]:
#DICTIONARY OF PRESELECTED EQUIPMENT BASED ON THE 'VENUE' THAT THE USER SELECTS. (KEY: Venue, VALUES: equipment)

gym_equipment = {
    "Fully equipped gym": {
        "equipment": [
            "2 Ankle strap",
            "1 Ankle strap",
            "2 Dumbbell",
            "1 Dumbbell",
            "2 Kettlebell",
            "1 Kettlebell",
            "2 Single grip handle",
            "1 Single grip handle",
            "45-degree leg press machine",
            "Adjustable pulley",
            "Assisted dip machine",
            "Assisted pull up machine",
            "Back extension station",
            "Bench",
            "Chest supported T-bar",
            "Curl bar",
            "Decline bench",
            "Dip machine",
            "EZ curl bar",
            "Fixed weight bar",
            "Flat chest press machine",
            "Functional trainer cable machine",
            "Hack squat machine",
            "Hex trap bar",
            "High pulley",
            "Horizontal leg press machine",
            "Incline bench",
            "Incline chest press machine",
            "Landmine base",
            "Lat pulldown cable machine",
            "Low pulley",
            "Lying down hamstring curl machine",
            "Mini loop band",
            "None",
            "Olympic barbell",
            "PVC pipe",
            "Parallel bars",
            "Pec deck machine",
            "Plate loaded lat pull down machine",
            "Plated row machine",
            "Platform",
            "Plyometric box",
            "Power tower",
            "Preacher bench",
            "Pull up bar",
            "Pull up station",
            "Pullover machine",
            "Quad extension machine",
            "Rope",
            "Seated abduction machine",
            "Seated adduction machine",
            "Seated cable pec fly machine",
            "Seated cable row machine",
            "Seated chest press machine",
            "Seated hamstring curl machine",
            "Seated lateral raise machine",
            "Seated overhead tricep extension machine",
            "Seated plated calf machine",
            "Seated shoulder press machine",
            "Seated tricep extension machine",
            "Smith machine",
            "Stability ball",
            "Standing lateral raise machine",
            "Standing plated calf machine",
            "Straight bar",
            "TRX",
            "Triceps V-bar",
            "Weight plates"
            ],

        "available_weights": {
            "Dumbbells":  {5:2, 7.5:2, 10:2, 12.5:2, 15:2, 17.5:2, 20:2, 25:2, 30:2, 35:2, 40:2, 45:2, 50:2,
                           55:2, 60:2, 65:2, 70:2, 75:2, 80:2, 85:2, 90:2, 95:2, 100:2},

            "Kettlebells": {5:2, 10:2, 15:2, 20:2, 25:2, 30:2, 35:2, 40:2, 45:2, 50:2},

            "Fixed weight bar": {10:1, 15:1, 20:1, 25:1, 30:1, 35:1, 40:1, 45:1, 50:1, 55:1, 60:1, 65:1, 70:1, 75:1, 80:1, 85:1, 90:1, 95:1, 100:1},

            "Mini loop band": {"Extra Light":1, "Light":1, "Medium":1, "Heavy":1, "Extra Heavy":1}

            },

    },

    "Moderately equipped gym":{
        "equipment": [
            "2 Ankle strap",
            "1 Ankle strap",
            "2 Dumbbell",
            "1 Dumbbell",
            "2 Single grip handle",
            "1 Single grip handle",
            "Bench",
            "Curl bar",
            "Decline bench",
            "EZ curl bar",
            "Fixed weight bar",
            "Functional trainer cable machine",
            "Horizontal leg press machine",
            "Incline bench",
            "Lat pulldown cable machine",
            "Lying down hamstring curl machine",
            "Mini loop band",
            "None",
            "Olympic barbell",
            "Platform",
            "Plyometric box",
            "Pull up station",
            "Quad extension machine",
            "Rope",
            "Seated cable row machine",
            "Seated chest press machine",
            "Smith machine",
            "Straight bar",
            "Triceps V-bar",
            "Weight plates"
            ],

        "available_weights": {
            "Dumbbells":  {5:2, 7.5:2, 10:2, 12.5:2, 15:2, 17.5:2, 20:2, 25:2, 30:2, 35:2, 40:2, 45:2, 50:2,
                           55:2, 60:2},

            "Fixed weight bar": {10:1, 20:1, 30:1, 40:1, 50:1, 60:1},

            "Mini loop band": {"Extra Light":1, "Light":1, "Medium":1, "Heavy":1, "Extra Heavy":1}

            }
    },

    "Home gym": {
        "equipment": [
            "2 Ankle strap",
            "1 Ankle strap",
            "2 Dumbbell",
            "1 Dumbbell",
            "2 Kettlebell",
            "1 Kettlebell",
            "2 Loop band",
            "1 Loop band",
            "2 Single grip handle",
            "1 Single grip handle",
            "Adjustable pulley",
            "Assisted dip machine",
            "Curl bar",
            "Decline bench",
            "EZ curl bar",
            "Handle band",
            "Incline bench",
            "Landmine base",
            "Mini loop band",
            "None",
            "Olympic barbell",
            "Parallel bars",
            "Platform",
            "Plyometric box",
            "Pull up bar",
            "Resistance band bar",
            "Rope",
            "Stability ball",
            "Straight bar",
            "TRX",
            "Triceps V-bar",
            "Weight plates"
            ],

        "available_weights": {
            "Dumbbells":  {5:2, 7.5:2, 10:2, 12.5:2, 15:2, 17.5:2, 20:2, 25:2, 30:2, 35:2, 40:2, 45:2, 50:2,
                           55:2, 60:2},

            "Kettlebells": {5:2, 10:2, 15:2, 20:2, 25:2, 30:2, 35:2, 40:2, 45:2, 50:2},

            "Mini loop band": {"Extra Light":1, "Light":1, "Medium":1, "Heavy":1, "Extra Heavy":1},

            "Loop band": {"Extra Light":2, "Light":2, "Medium":2, "Heavy":2, "Extra Heavy":2},

            "Handle band": {"Extra Light":1, "Light":1, "Medium":1, "Heavy":1, "Extra Heavy":1}

            }
    },



    "Minimal equipment setup": {
        "equipment": [
            "2 Ankle strap",
            "1 Ankle strap",
            "2 Dumbbell",
            "1 Dumbbell",
            "2 Loop band",
            "1 Loop band",
            "2 Single grip handle",
            "1 Single grip handle",
            "Handle band",
            "Mini loop band",
            "None",
            "Platform",
            "Resistance band bar",
            "Rope",
            "Stability ball"
            ],

        "available_weights": {
            "Dumbbells":  {5:2, 7.5:2, 10:2, 12.5:2, 15:2, 17.5:2, 20:2, 25:2, 30:2, 35:2},

            "Mini loop band": {"Extra Light":1, "Light":1, "Medium":1, "Heavy":1, "Extra Heavy":1},

            "Loop band": {"Extra Light":2, "Light":2, "Medium":2, "Heavy":2, "Extra Heavy":2},

            "Handle band": {"Extra Light":1, "Light":1, "Medium":1, "Heavy":1, "Extra Heavy":1}
        }
    },

    "No setup": {
        "equipment": ["None"],
        "available_weights": {},
    }
}

In [5]:
import random

#Utility function to determine exercise distribution based on ranges and cosidering priority muscles.

def generate_biased_distribution(ranges, priority_muscles, muscles, bias_factor=0.1):
    """
    Generate a randomized distribution with a fixed bias for priority muscles.

    Parameters:
    - ranges (list of tuples): (min, max) range for each muscle.
    - priority_muscles (list): List of priority muscles to bias towards.
    - muscles (list): List of muscle names corresponding to ranges.
    - bias_factor (float): Bias proportion to apply (e.g., 0.1 for 10%).

    Returns:
    - List of randomized distribution values.
    """
    total_sum = 1.0
    bias_increment = total_sum * bias_factor  # 10% of the total sum
    distribution = []
    remaining_sum = total_sum

    # Generate initial random values
    for i, (min_val, max_val) in enumerate(ranges):
        muscle = muscles[i]
        if i == len(ranges) - 1:
            value = remaining_sum
        else:
            value = round(random.uniform(min_val, min(max_val, remaining_sum)), 2)
            remaining_sum -= value
        
        # Apply bias increment to priority muscles
        if muscle in priority_muscles:
            value += bias_increment
            #value = min(value, max_val)  # Ensure it doesn't exceed the upper bound (keep commented)
        
        distribution.append(value)

    # Normalize the distribution to ensure sum equals 1.0
    total = sum(distribution)
    normalized_distribution = [round(x / total, 2) for x in distribution]

    return normalized_distribution

# HELPER FUNCTIONS TO RECOMMEND SPLIT

## FROM recommend_split.py

In [6]:
#HELPER FUNCTION TO DETERMINE IF A VALID COMBINATION OF DAYS EXIST FOR 3-DAY FULL BODY
#days / days_of_week are arrays from 1-7, representing the days of the week.

def valid_fullbody_gaps(days):
    #find combinations of three days
    for day1, day2, day3 in combinations(days, 3):
        gap1 = day2 - day1
        gap2 = day3 - day2
        gap3 = (7 - day3) + day1 

        if gap1 >= 2 and gap2 >= 2 and gap3 >= 2:
            return True
    
    return False

#FUNCTION TO DETERMINE IF THE GAPS ARE VALID FOR A 4-DAY SPLIT (UPPER-LOWER, PHUL) 

def has_valid_gaps(days_of_week):
    
    # Check all combinations of two days
    for day1, _, day3, day4 in combinations(days_of_week, 4):
        gap1 = day3 - day1  # Direct difference
        gap2 = (7 - day4) + day1  # Wrap-around difference
        
        # Check if both directions respect the minimum difference
        if gap1 >= 3 and gap2 >= 3: 
            return True  # Valid pair found
    
    return False  # No valid pairs found

#FUNCTION TO CHECK IF THE USER SCHEDULE ALLOWS FOR A 2-DAY SPLIT (FULL-BODY) OR A 3-DAY SPLIT (UPPER-LOWER)

def has_min_gap(days_of_week, min_difference):
    
 # (min_difference - 1) tell us the minimum number of days between workouts.

    # Check all combinations of two days
    for day1, day2 in combinations(days_of_week, 2):
        direct_gap = abs(day2 - day1)  # Direct difference
        wrap_around_gap = (7 - max(day1, day2)) + min(day1, day2)  # Wrap-around difference
        
        # Check if both directions respect the minimum difference
        if direct_gap >= min_difference and wrap_around_gap >= min_difference:
            return True  # Valid pair found
    
    return False  # No valid pairs found


# RECOMMEND SPLIT FUNCTION
## FROM recommend_split.py

In [7]:
#LOGIC BEHING THE recommend_split() function.

# days_of_week: array [1-7], where 1: Monday, 2: Tuesday ...
# workout_frequency (1-7)
# time_per_workout: 30 min - 2 hours (30, 45, 60, 75, 90, 105, 120)  

#If wf == 1, Full body split 

    #If wf == 2, full body(f=2) OR Upper/lower(f=1)
        #Full body: Largest difference between days_of_week must be at least 3 for newcomers (2 rest days), can be 2 for begginers + (1 rest day)
        #Else, Upper/lower

    #If wf == 3, full body(f=3) OR Upper lower(f=1.5), PPL(f=1)
         #Upper lower: Appropiate for everyone. At least One largest difference between days_of_week must be 3 (2 days in between)
        #Full body: Largest difference between days_of_week must be 2 (1 day) twice for beginners ++
        #Else: PPL

    #If wf == 4, PHUL (f=2), upper lower(f=2), PPL(f=1.33)
        #PHUL if want 50/50 strengh and muscle AND can do a rest day after 2 workouts
        #Upper/lower if can do a rest day after 2 workout
        #else PPL

    #If wf == 5, [upper lower(f=2.5),  PHUL (f=2.5)]
    #Hybrid PPL + Upper/Lower (f=2), PPL(f=1.66), Body part split(f=1)

    #If wf == 6,  [upper lower(f=3), PHUL (f=3)]
    # PPL(f=2), but 6-day body plat split (f=1) if limited time
     
    #If wf == 7,  [upper lower(f=3), PHUL (f=3)] + active rest
    #PPL(f=2) + active rest

    #days selected MUST BE EQUAL OR LARGER than WORKOUT 

# split_dictionary = {
#     "Upper-lower"    : [["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Abs","Obliques", "Lower back"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"]],

#     "Push-Pull-Legs" : [["Chest","Shoulders", "Triceps"],
#                         ["Back", "Biceps","Trapezius", "Abs","Obliques"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves","Lower back"]],

#     "PHUL"           : [["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Abs","Obliques", "Lower back"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"]],
    
#     "Hybrid PPL+ UL" : [["Chest","Shoulders", "Triceps"],
#                         ["Back", "Biceps","Trapezius", "Abs","Obliques"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves","Lower back"],
#                         ["Chest","Shoulders", "Triceps", "Back", "Biceps","Trapezius", "Abs","Obliques", "Lower back"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"]],

#     "Body part split": [["Chest"],
#                         ["Back","Trapezius"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves", "Lower back"],
#                         ["Shoulders","Abs","Obliques"],
#                         ["Biceps","Triceps"]],

#     "6-day body parysplit": [["Chest"],
#                         ["Back","Trapezius"],
#                         ["Quads", "Glutes", "Hamstrings", "Abductors","Adductors","Calves"],
#                         ["Abs","Obliques","Lower back"],
#                         ["Shoulders"],
#                         ["Biceps","Triceps"]]
# }

In [8]:

def recommend_split (days_of_week, workout_frequency, time_per_workout, level, goals):

    if workout_frequency == 1:
        return 'Full-Body'
    
    if workout_frequency == 2:
        
        if has_min_gap(days_of_week,3) and level == 0:
            return 'Full-Body'
        if has_min_gap(days_of_week,2) and level >= 1:
            return 'Full-Body'
        else:
            return 'Upper-Lower'
        
    if workout_frequency == 3:
  
        if has_min_gap(days_of_week,3) and time_per_workout > 45:
            return 'Upper-Lower'
        if valid_fullbody_gaps(days_of_week) and level >= 1 and time_per_workout <= 45:
            return 'Full-Body' #MIGHT CHANGE THIS!!!
        else:
            return 'Push-Pull-Legs'
        
    if workout_frequency == 4:
        valid = has_valid_gaps(days_of_week)
        if any(goal in ['Powerlifting', 'Get stronger'] for goal in goals) and any(goal in ['Bodybuilding', "Build Muscles", "Get Lean"] for goal in goals) and valid:
            return 'PHUL' #IN THIS ESCENARIO, WE SHOULD PRIORITIZE STRENGHT A BIT MORE. 
        if valid:
            return 'Upper-Lower'
        else:
            return 'Push-Pull-Legs'
        
    if workout_frequency == 5: 
        #valid = has_valid_gaps(days_of_week)
        #MIGHT CHANGE THE FOLLOWING IF STATEMENT, AND MODIFY HYBRID PPL + Upper/Lower logic
        if any(goal in ['Powerlifting', 'Get stronger'] for goal in goals) and any(goal in ['Bodybuilding', "Build Muscles", "Get Lean"] for goal in goals):
            return 'Hybrid PPL + Upper/Lower'
        if time_per_workout < 45:
            return 'Body part split'
        else:
            return 'Push-Pull-Legs'

    if workout_frequency == 6:
        if time_per_workout <= 30:
            return '6-day body part split'
        return 'Push-Pull-Legs'
    
    
    if workout_frequency == 7:
        return 'Push-Pull-Legs + active rest' 

WHAT IF THE USER WANTS A CUSTOM SPLIT? IN OTHER WORDS, ONE WHERE HE/SHE DECIDES THE GROUPING. HOW WE DETERMINE THE WEIGHTS? DO WE NEED THIS FEATURE?

# COPIES OF FILTER DATA FUNCTIONS

## FROM filtering_dataset.ipynb

THIS IS THE FIRST VERSION OF FILTER_DATA() FUNCTION. IT IS COMMENTED BECAUSE AN OPTIMIZED VERSION FOLLOWS.

In [9]:
# def filter_data(df, user_level, user_equipment, training_modalities, pain_points, age):
#     """
#     Filters exercises based on user level, equipment, training modalities, pain points, and age.

#     Parameters:
#     - user_level (str/int): User's experience level.
#     - user_equipment (list): List of equipment available to the user.
#     - training_modalities (list): List of desired training modalities.
#     - pain_points (list): List of user's pain points.
#     - age (int): User's age.

#     Returns:
#     - filtered_df (DataFrame): Filtered Pandas DataFrame based on user criteria.
#     """
    
#     # Adjust training modalities if user is at level 0
#     if user_level == 0:
#         training_modalities.append('MP')
    
#     filtered_df = df.copy()  # Start with the full dataset to avoid overwriting the original
    
#     # 1. Filter by Experience Level
#     filtered_df = filtered_df[
#         filtered_df['Level'].apply(lambda x: user_level in x)
#     ]
    
#     # User-provided equipment set
#     user_equipment_set = set(user_equipment)
#     print(user_equipment_set)

#     # Equipment Filtering Logic
#     filtered_df = filtered_df[
#         filtered_df['Equipment'].apply(
#             lambda subsets: any(
#                 set(subset).issubset(user_equipment_set) for subset in subsets if subset
#             )
#         )
#     ]
    
#     # 3. Filter by Training Modalities
#     filtered_df = filtered_df[
#         filtered_df['Exercise Purpose'].apply(
#             lambda x: any(modality in x for modality in training_modalities)
#         )
#     ]
    
#     # 4. Filter by Risk Level (if user is older than 50)
#     if age > 50:
#         filtered_df = filtered_df[
#             filtered_df['Risk level'] <= 2
#         ]
    
#     # 5. Exclude Exercises Based on Pain Points
#     filtered_df = filtered_df[
#         ~filtered_df['Pain Exclusions'].apply(
#             lambda x: any(pain in x for pain in pain_points)
#         )
#     ]
    
#     return filtered_df

# OPTIMIZED VERSION

## FROM filtering_dataset.ipynb

In [10]:
def filter_data(df, user_level, user_equipment, training_modalities, age, pain_points):
    """
    Filters exercises based on user level, equipment, training modalities, pain points, and age.

    Parameters:
    - df (DataFrame): The dataset to filter.
    - user_level (str/int): User's experience level.
    - user_equipment (list): List of equipment available to the user.
    - training_modalities (list): List of desired training modalities.
    - pain_points (list): List of user's pain points.
    - age (int): User's age.

    Returns:
    - filtered_df (DataFrame): Filtered Pandas DataFrame based on user criteria.
    """
    # Ensure user equipment is a set for faster lookups
    user_equipment_set = set(user_equipment)
    
    # Apply filters
    filtered_df = df[
        (df['Level'].apply(lambda x: user_level in x)) &
        (df['Equipment'].apply(
            lambda subsets: any(
                set(subset).issubset(user_equipment_set) for subset in subsets if subset
            )
        )) &
        (df['Exercise Purpose'].apply(lambda x: any(modality in x for modality in training_modalities))) &
        (~df['Pain Exclusions'].apply(lambda x: any(pain in x for pain in pain_points))) &
        ((age <= 50) | (df['Risk level'] <= 2))
    ]
    
    return filtered_df


# SECONDARY FILTERING FUNCTIONS

## FROM filtering_dataset.ipynb

In [11]:
def filter_muscles(df, muscles):
    """
    Filters exercises based on muscle.

    Parameters:
    - df (DataFrame): The dataset to filter.
    - muscles (list): Keep these muscles on the dataset

    Returns:
    - filtered_df (DataFrame): Filtered Pandas DataFrame based on user criteria.
    """
    # Apply filters
  # 3. Filter by muscles
    filtered_df = df[
        df['Main muscle(s)'].apply(
            lambda x: any(muscle in x for muscle in muscles)
        )
    ]
    
    return filtered_df

In [12]:
#FUNCTION TO CALCULATE TOTAL NUMBER OF EXERCISES FOR THE WORKOUT BASED ON:

#CAN BE CHANGE BY THE USER:
# time_for_lifting (May be different than total workout time). 
# rest_time_per_set

#CONSTANTS THAT WE PASS:
# avg_time_per_set
# sets_per_exercise


def calculate_total_exercises(time_for_lifting, avg_time_per_set, sets_per_exercise, rest_time_per_set):
    time_per_exercise = (avg_time_per_set + rest_time_per_set) * sets_per_exercise

    total_exercises = time_for_lifting // time_per_exercise

    return total_exercises

In [13]:
#FUNCTION TO CALCULATE NUMBER OF EXERCISES PER MUSCLE GROUP FOR THE SPECIFIC WORKOUT SESSION:

def calculate_exercises_per_muscle(total_exercises, muscle_groups, focus_distribution):
    
    num_muscle_groups = len(muscle_groups)

    if len(focus_distribution) != num_muscle_groups:
        raise ValueError("Focus distribution must match the number of muscle groups.")

    exercises_per_muscle = {
        muscle: round(total_exercises * focus)
        for muscle, focus in zip(muscle_groups, focus_distribution)
    }

    return exercises_per_muscle

# UNIQUE IN THIS FILE

In [14]:
import random

def select_exercises(filtered_df, exercises_per_muscle):
    """
    Randomly select exercises for each muscle group.

    Parameters:
    - filtered_df (DataFrame): Filtered dataset containing exercises.
    - exercises_per_muscle (dict): Dictionary with the number of exercises to select per muscle group.

    Returns:
    - dict: Dictionary with selected exercises for each muscle group.
    """
    selected_exercises = {}

    for muscle, num_exercises in exercises_per_muscle.items():
        # Filter exercises for the current muscle group
        muscle_exercises = filtered_df[
            filtered_df['Main muscle(s)'].apply(lambda x: muscle in x)
        ]

        # Randomly sample exercises for this muscle group
        if num_exercises > len(muscle_exercises):
            print(f"Warning: Not enough exercises for {muscle}. Selecting all available.")
            selected_exercises[muscle] = muscle_exercises  # Select all if not enough
        else:
            selected_exercises[muscle] = muscle_exercises.sample(n=num_exercises)

        selected_exercises[muscle] = selected_exercises[muscle].sort_values(by="Difficulty", ascending=False)
        

    return selected_exercises


In [15]:
import random 

def new_select_exercises(filtered_df, exercises_per_muscle, user_favorites=None, suggest_less=None, dont_show_again=None):
    """
    Select exercises for each muscle group based on preferences.

    Parameters:
    - filtered_df (DataFrame): Filtered exercise dataset.
    - exercises_per_muscle (dict): Number of exercises per muscle.
    - user_favorites (set): Exercises to prioritize.
    - suggest_less (set): Exercises to deprioritize.
    - dont_show_again (set): Exercises to exclude completely.

    Returns:
    - dict: Selected exercises per muscle group.
    """

    FAVORITE_WEIGHT = 2
    SUGGEST_LESS_WEIGHT = 0.25

    user_favorites = user_favorites or set()
    suggest_less = suggest_less or set()
    dont_show_again = dont_show_again or set()

    selected_exercises = {}

    for muscle, num_exercises in exercises_per_muscle.items():
        #First, filter exercises for the muscle group
        muscle_exercises = filtered_df[
            filtered_df["Main muscle(s)"].apply(lambda x: muscle in x)
            ]
        
        #Remove "Don't show again" exercises
        muscle_exercises = muscle_exercises[
            ~muscle_exercises["Exercise"].isin(dont_show_again)
        ]

        #Assign sampling weights
        exercise_weights = muscle_exercises["Exercise"].apply(
            lambda x: FAVORITE_WEIGHT if x in user_favorites
            else SUGGEST_LESS_WEIGHT if x in suggest_less
            else 1
        )

        #Sample exercises
        if num_exercises > len(muscle_exercises):
            print(f"Warning: Not enough exercises for {muscle}. Selecting all available.")
            sampled = muscle_exercises
        else:
            sampled = muscle_exercises.sample(
                n=num_exercises, weights=exercise_weights, replace=False
            )

        #Sort by difficulty
        sampled = sampled.sort_values(by="difficulty", ascending=False)
        selected_exercises[muscle] = sampled

    return selected_exercises

In [16]:
#CURRENTLY INITIALIZED GLOBALLY. MAPPING FROM GOAL TO MODALITY. USED TO DETERMINE PHASES AND CORRESPONDING REP RANGE.

goal_to_training_phase = {
    "Get stronger": "Strength",
    "Bodybuilding": "Hypertrophy",
    "Build muscles": "Hypertrophy",
    "Aesthetics": "Hypertrophy",
    "Losing weight": "Hypertrophy",
    "Get lean": "Hypertrophy",
    "Increase endurance": "Endurance"
}

In [17]:

def get_training_phase_and_group_for_day(user_goals, user_split, current_day):
    """
    Determines which unique training phase should be used for a workout session.

    Parameters:
    - user_goals (list): Ordered list of user-selected goals.
    - split_structure (string): The recommended workout split (e.g. push-pull-legs).
    - current_day (int): The workout day count (0-indexed).

    Returns:
    - tuple: (training_phase, workout_focus)
    """

    #convert user goals into unique training phases
    unique_phases = list({goal_to_training_phase[goal] for goal in user_goals})

    #Find how many days make up a full cycle of the split
    split_length = len(split_dictionary_complex[user_split]["groups"])
    #print(split_length)

    #Determine the index of the current split day (i.e. 0 for push, 1 for pull, etc).
    split_muscle_group_index = current_day % split_length

    #Determine training phase (strenght: low reps, high rest time - Hyperthrophy: medium reps, moderate rest time. etc)
    training_phase = unique_phases[(current_day // split_length) % len(unique_phases)]
    #print(training_phase)

    return training_phase, split_muscle_group_index

In [18]:
# #CAN BE IGNORE. JUST TESTING...

# user_goals = ["Get stronger", "Build muscles", "Aesthetics", "Losing weight"]
# user_split = "Push-Pull-Legs"

# for day in range(10):  # Simulating 10 workout days
#     training_phase, muscle_group_index = get_training_phase_and_group_for_day(user_goals, user_split, day)
#     muscle_group = split_dictionary_complex[Jose_split]["groups"][muscle_group_index]
#     print(f"Day {day+1}: {muscle_group} | Training Phase: {training_phase}")

In [19]:
def get_reps_and_rest_time(training_phase):
    """
    Returns the rep range and rest time based on the training phase.

    Parameters:
    - training_phase (str): The current training phase.

    Returns:
    - dict: {"reps": (min_reps, max_reps), "rest_time": rest_time} or None if invalid.
    """
    phase_to_reps_and_time = {
        "Strength": {"reps": (5, 7), "rest_time": 3},
        "Hypertrophy": {"reps": (8, 12), "rest_time": 2},
        "Endurance": {"reps": (13, 16), "rest_time": 2}
    }
    return phase_to_reps_and_time.get(training_phase)


In [20]:
# #CAN BE IGNORE. JUST TESTING...

# # Example Usage:
# for day in range(10):  
#     phase,  muscle_group_index = get_training_phase_and_group_for_day(user_goals, user_split, day)
#     muscle_group = split_dictionary_complex[Jose_split]["groups"][muscle_group_index]
#     reps, rest_time = get_reps_and_rest_time(phase)["reps"], get_reps_and_rest_time(phase)["rest_time"]
    
#     print(f"Day {day+1}: {muscle_group} | Phase: {phase} | Reps: {reps} | Rest Time: {rest_time} min")

In [21]:
def round_gym_weight(weight, is_pair=False):

    """
    Rounds weight to the nearest standard gym equipment.
    If is_pair=True, it means it is dumbbell or kettlebell.
    It returns in terms of TOTAL WEIGHT
    """
    common_weights = [5, 7.5, 10, 12.5, 15, 17.5, 20, 22.5, 25, 30, 35, 40, 45, 50,
                        55, 60, 65, 70, 75, 80, 85, 90, 95, 100]
    
    if is_pair:
        per_piece = weight / 2
        closest = min(common_weights, key=lambda x: abs(x - per_piece))
        return closest * 2 
        #NOTE: Shouldn' I just return closest (which is the weight for a dumbbell or ketlebell)
    
    else:
        return round(weight / 5) * 5  # Ensures valid weight increments

In [22]:
#########################################################################################

#THIS IS A ANOTHER VARIATION OF THE INFER_EQUIPMENT_TYPE, JUST IN CASE THE OTHER ONE DOESN'T WORK

# def infer_equipment_type(lower_bound):
#     """Infers equipment type based on the format of the lower bound value."""
#     if lower_bound == 0:
#         return "Bodyweight"
#     elif isinstance(lower_bound, (int, float)):
#         return "Gym Equipment"  # Free weights, machines
#     elif isinstance(lower_bound, str) and lower_bound.replace(" ", "").isdigit():
#         return "Timed Exercise"  # Example: "30 seconds"
#     elif isinstance(lower_bound, str):
#         return "Resistance Band"  # Example: "Light", "Medium", "Heavy"
#     return "Unknown"

In [23]:
#TO NARROW THE FLOW OF INFORMATION, INSTEAD OF PASSING THE WHOLE 'RECORDS" WE COULD JUST PASS THE SPECIFIC USER'S RECORD!!! TO NARROW DOWN THE FLOW OF INFORMATION

#How do we want to format the DATE?

#THIS FUNCTION HASN'T BEEN TESTED, BUT CAN BE USED TO RETRIEVE RECORDS BY DATE

#################################################################################
def get_workout_by_date(records, user_id, date):
    """
    Retrieves a full workout session for a given user on a specific date.

    Parameters:
    - records (dict): The workout records.
    - user_id (str): Unique user ID.
    - date (str): The date of the workout session (format: YYYY-MM-DD).

    Returns:
    - list or None: List of exercises for that date or None if not found.
    """
    if user_id in records and date in records[user_id]["by_date"]:
        return records[user_id]["by_date"][date]
    else:
        return None
    
##################################################################################

In [24]:
# Resistance band levels
band_progression = ["Extra Light", "Light", "Medium", "Heavy", "Extra Heavy"]

In [25]:
def infer_equipment_type(min_weight):
    if isinstance(min_weight, (int, float)) and min_weight > 0:
        return "Gym Equipment"
    elif isinstance(min_weight, str):
        if "seconds" in min_weight:
            return "Timed Exercise"
        elif min_weight in band_progression:
            return "Resistance Band"
    return "Bodyweight"

In [26]:
#THE CURRENT FORMAT OF THE LIST OF EXERCISES, CALLED 'EXERCISES' IS key: Str (i,e. Chest), Val: DataFrame. SHOULD WE CHANGE IT????

# if isinstance(exercises, list):
#     print("exercises is a list")
#     if exercises:
#         print(f"First item type: {type(exercises[0])}")
# elif isinstance(exercises, dict):
#     print("exercises is a dictionary")
#     print(f"Keys type: {type(next(iter(exercises.keys())))}")
#     print(f"Key: {(next(iter(exercises.keys())))}")
#     print(f"Values type: {type(next(iter(exercises.values())))}")
#     print(f"Value: {(next(iter(exercises.values())))}")

In [27]:
def estimate_weight_Brzycki(actual_weight, actual_reps, target_reps):
    """
    Estimates the weight needed to achieve a specific number of target reps using the Brzycki formula.

    Parameters:
    - actual_weight (float): The weight previously lifted.
    - actual_reps (int): The number of reps performed with that weight.
    - target_reps (int): The desired number of reps for the next session.

    Returns:
    - float: Suggested weight for the target reps.
    """

    # Ensure reps are within a valid range for the Brzycki formula
    if actual_reps > 10 or target_reps > 10:
        raise ValueError("Brzycki formula is most accurate for reps <= 10.")

    # Step 1: Estimate 1RM using Brzycki formula
    one_rep_max = actual_weight / (1.0278 - (0.0278 * actual_reps))

    # Step 2: Calculate weight for the target reps
    target_weight = one_rep_max * (1.0278 - (0.0278 * target_reps))

    # Round to the nearest 5 lbs (for gym-friendly increments)
    target_weight = round(target_weight / 5) * 5

    return target_weight


In [28]:
def estimate_weight_epley(actual_weight, actual_reps, target_reps):
    """
    Estimates the weight needed to achieve a specific number of target reps using the Epley Formula.

    Parameters:
    - actual_weight (float): The weight previously lifted.
    - actual_reps (int): The number of reps performed with that weight.
    - target_reps (int): The desired number of reps for the next session.

    Returns:
    - float: Suggested weight for the target reps.
    """

    # Step 1: Estimate 1RM using Epley formula
    one_rep_max = actual_weight * (1 + 0.0333 * actual_reps)

    # Step 2: Estimate weight for the target reps
    target_weight = one_rep_max / (1 + 0.0333 * target_reps)

    # Step 3: Round to nearest 5 lbs for gym equipment compatibility
    target_weight = round(target_weight / 5) * 5

    return target_weight


In [29]:
import math 

def weight_algorithm(user_exercise_records, phase):
    """
    Suggests weight and reps.

    Parameters:
    - user_exercise_records (list): User's past records for this exercise.
    - phase (str): Current training phase (Strength, Hypertrophy, Endurance).

    Returns:
      - dict: {"weight": suggested_weight, "reps": [rep1, rep2, rep3]}
    """

    # Define rep ranges and progression rules per phase
    phase_settings = {
        "Strength": {"target_reps": (5, 8), "increment": 10},
        "Hypertrophy": {"target_reps": (9, 12), "increment": 10},
        "Endurance": {"target_reps": (13, 16), "increment": 10}
    }

    settings = phase_settings[phase]
    target_reps = settings["target_reps"]
    increment = settings["increment"]

    phase_records = [rec for rec in user_exercise_records if rec["phase"] == phase]
    #history = {(rec["weight"], rec["reps"]) for rec in phase_records}               #HOW MANY RECORDS SHOULD WE KEEP?   
    history = {(rec["weight"], tuple(rec["reps"])) for rec in phase_records}

    def has_better_performance(weight, suggested_reps):
        for rec_weight, rec_reps in history:
            if rec_weight == weight and math.floor(sum(rec_reps)) >= suggested_reps *len(rec_reps):
                return True
        return False


    if not phase_records:
        #call either Brzycki or epley function
            # Get the most recent record
        last_record = user_exercise_records[-1]
        last_weight = last_record["weight"]
        last_reps = last_record["reps"]
        avg_reps = sum(last_reps) / len(last_reps)  # Example: (9+9+8)/3 = 8.67
        avg_reps = round(avg_reps)  # Optional: round to 9

        if target_reps[0] <= 10:
            # Use Brzycki for Strength & Hypertrophy
            estimated_weight = estimate_weight_Brzycki(last_weight, avg_reps, target_reps[0])
        else:
            # Use Epley for Endurance
            estimated_weight = estimate_weight_epley(last_weight, avg_reps, target_reps[0])

        return {"weight": estimated_weight, "reps": target_reps[0]}
    

    last_record = phase_records[-1]
    last_weight = last_record["weight"]
    last_reps = last_record["reps"]
    avg_reps = sum(last_reps) / len(last_reps)  # Example: (9+9+8)/3 = 8.67
    avg_reps = round(avg_reps)  # Optional: round to 9

     # 1️⃣ Try decreasing weight and increasing reps (+2 reps if within range)
    if avg_reps + 2 <= target_reps[1]:
        decreased_weight = last_weight - increment
        new_combo = (decreased_weight, avg_reps + 2)

        if new_combo not in history and not has_better_performance(new_combo[0], new_combo[1]):
            #return new_combo
            return {"weight": decreased_weight, "reps": avg_reps + 2}
        
    # 2️⃣ If Decrease-Weight/Increase-Reps is Not Possible, Increase Weight
    increased_weight = last_weight + increment

    # Option A: Reduce reps by 2 from the last record 
    if avg_reps - 2 >= target_reps[0] and avg_reps - 2 <= target_reps[1]:
        new_combo = (increased_weight, avg_reps - 2)
        if new_combo not in history and not has_better_performance(new_combo[0], new_combo[1]):
            #return new_combo
            return {"weight": increased_weight, "reps": avg_reps - 2}
        new_combo = (increased_weight, avg_reps - 1)
        if new_combo not in history and not has_better_performance(new_combo[0], new_combo[1]):
            #return new_combo
            return {"weight": increased_weight, "reps": avg_reps - 1}
        
    # 3️⃣ Final Fallback: Maintain weight and increase reps by 1 if within range
    if avg_reps + 1 <= target_reps[1]:
        new_combo = (last_weight, avg_reps + 1)
        if new_combo not in history and not has_better_performance(new_combo[0], new_combo[1]):
            #return new_combo
            return {"weight": last_weight, "reps": avg_reps + 1}
        
     # If all else fails, introduce a new heavier weight and start from the lower rep range
    #return increased_weight, target_reps[0]
    return {"weight": increased_weight, "reps": target_reps[0]}


In [30]:
#TODO: WE DO HAVE BODYWEIGHT EXERCISES MAP TO 'PROGESSION EXERCISES' in the dataset! If upper bound reach, we could suggest progression

#SHOULD WE NOT CARE ABOUT THE PHASE IN THIS CASE? IT SEEMS COUNTERPRODUCTIVE TO DO 5-8 REPS.

def bodyweight_algorithm(exercise_records, phase):
    # Define rep ranges
    phase_settings = {
        "Strength": {"target_reps": (5, 8)},
        "Hypertrophy": {"target_reps": (9, 12)},
        "Endurance": {"target_reps": (13, 16)}
    }

    settings = phase_settings[phase]
    target_reps = settings["target_reps"]

    phase_records = [rec for rec in exercise_records if rec["phase"] == phase]

    if not phase_records:
        #return target_reps[0]
        return {"weight": None, "reps": target_reps[0]}
    
    # Get the most recent record
    last_record = phase_records[-1]
    last_reps = last_record["reps"]

    # Base progression on the minimum reps completed across sets
    min_reps = min(last_reps)

    # Increase reps if possible
    if min_reps < target_reps[1]:
        #return min_reps + 1
         return {"weight": None, "reps": min_reps + 1}
    else:
        #return target_reps[1]
        return {"weight": None, "reps": target_reps[1]}


In [31]:
def test_bodyweight_algorithm_varied():
    """
    Test the bodyweight_algorithm function with varied user performance data.
    """
    simulated_records = []
    reps_list = [[9, 9, 9], [10, 10, 10], [11, 11, 11], [12, 12, 12], [10, 10, 10]]

    for reps in reps_list:
        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": None,
            "reps": reps
        })

    for i in range(10):
        result = bodyweight_algorithm(simulated_records, "Hypertrophy")
        
        # Simulate user performance with slight variation
        user_performance = [max(result['reps'] - random.randint(0, 2), 8) for _ in range(3)]
        
        print(f"Day {i + 1}: Suggested Reps: {result['reps']}, User Performance: {user_performance}")

        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": None,
            "reps": user_performance
        })

# Run the test
test_bodyweight_algorithm_varied()



Day 1: Suggested Reps: 11, User Performance: [10, 10, 11]
Day 2: Suggested Reps: 11, User Performance: [11, 11, 11]
Day 3: Suggested Reps: 12, User Performance: [10, 12, 12]
Day 4: Suggested Reps: 11, User Performance: [9, 9, 11]
Day 5: Suggested Reps: 10, User Performance: [8, 8, 8]
Day 6: Suggested Reps: 9, User Performance: [9, 8, 9]
Day 7: Suggested Reps: 9, User Performance: [8, 9, 9]
Day 8: Suggested Reps: 9, User Performance: [9, 8, 9]
Day 9: Suggested Reps: 9, User Performance: [8, 8, 9]
Day 10: Suggested Reps: 9, User Performance: [8, 9, 8]


In [32]:
def timed_algorithm(exercise_records):
    """
    Suggests time duration for timed exercises like planks, wall sits, etc.

    Parameters:
    - exercise_records (list): User's past records for this exercise.

    Returns:
    - int: Suggested duration (seconds) for the next workout.
    """

    # Default time settings
    max_time = 180  # Max cap at 3 minutes
    increment = 5  # Increase by 5 seconds if successful

    # If no records exist, start from the base time
    # if not exercise_records:
    #     return 30


    # Get the most recent record
    last_record = exercise_records[-1]
    last_times = last_record["time"]

    # Calculate the average time (since it's recorded as a list of times)
    avg_time = sum(last_times) // len(last_times)

    # Increase time if possible
    if avg_time < max_time:
        new_time = avg_time + increment
        #return min(new_time, max_time)
        return  {"weight": None, "reps": None, "time": min(new_time, max_time)}
    else:
        # Maintain the max time if it's already reached
        #return max_time
        return {"weight": None, "reps": None, "time": max_time}


In [33]:
def test_timed_algorithm():
    """
    Test the timed_algorithm function with simulated workout data.
    """
    simulated_records = []
    times_list = [[30, 30, 30], [35, 35, 35], [40, 40, 40], [45, 45, 45], [50, 50, 50]]

    for time in times_list:
        simulated_records.append({
            "phase": "Endurance",
            "weight": None,
            "time": time  # Storing time as reps for simplicity
        })

    for i in range(10):
        result = timed_algorithm(simulated_records)
        
        # Simulate user performance with slight variation (randomly adding/subtracting up to 5 seconds)
        user_performance = [max(result['time'] - random.randint(0, 5), 20) for _ in range(3)]
        
        print(f"Day {i + 1}: Suggested Time: {result['time']} seconds, User Performance: {user_performance}")

        simulated_records.append({
            "phase": "Endurance",
            "weight": None,
            "time": user_performance
        })

# Run the test
test_timed_algorithm()


Day 1: Suggested Time: 55 seconds, User Performance: [53, 54, 54]
Day 2: Suggested Time: 58 seconds, User Performance: [54, 55, 57]
Day 3: Suggested Time: 60 seconds, User Performance: [58, 60, 59]
Day 4: Suggested Time: 64 seconds, User Performance: [59, 62, 63]
Day 5: Suggested Time: 66 seconds, User Performance: [65, 65, 64]
Day 6: Suggested Time: 69 seconds, User Performance: [68, 69, 68]
Day 7: Suggested Time: 73 seconds, User Performance: [71, 71, 73]
Day 8: Suggested Time: 76 seconds, User Performance: [73, 76, 72]
Day 9: Suggested Time: 78 seconds, User Performance: [73, 75, 73]
Day 10: Suggested Time: 78 seconds, User Performance: [76, 78, 76]


In [34]:
def band_algorithm(exercise_records, phase):
    """
    Suggests band resistance progression based on past performance.

    Parameters:
    - exercise_records (list): User's past records for this exercise.
    - phase (str): Current training phase (Strength, Hypertrophy, Endurance).

    Returns:
    - str: Suggested resistance level for the next workout.
    """

    # Resistance band tension hierarchy (from lightest to heaviest)
    band_levels = ["Light", "Medium", "Heavy", "Extra Heavy"]

    # Phase settings to determine target rep ranges
    phase_settings = {
        "Strength": {"target_reps": (5, 8)},
        "Hypertrophy": {"target_reps": (9, 12)},
        "Endurance": {"target_reps": (13, 16)}
    }

    # Get the target rep range based on the phase
    settings = phase_settings[phase]
    target_reps = settings["target_reps"]

    # If no records exist, start with the lightest band
    # if not exercise_records:
    #     return band_levels[0]

    # Get the most recent record
    last_record = exercise_records[-1]
    last_band = last_record["weight"]  # Stores band level as string
    last_reps = last_record["reps"]
    avg_reps = sum(last_reps) // len(last_reps)

    # Find the index of the current band in the hierarchy
    if last_band not in band_levels:
        # Fallback to light if the recorded band doesn't match known bands
        last_band_index = 1
    else:
        last_band_index = band_levels.index(last_band)

    # 1️⃣ If reps exceed the upper limit → move to next resistance level
    if avg_reps >= target_reps[1] and last_band_index < len(band_levels) - 1:
        #return band_levels[last_band_index + 1]
        return {"weight": band_levels[last_band_index + 1], "reps": target_reps[0]}

    # 2️⃣ If reps are within target range → maintain band resistance but increase reps
    if target_reps[0] <= avg_reps < target_reps[1]:
        #return last_band
        return {"weight": last_band, "reps": avg_reps + 1}

    # 3️⃣ If reps are below target range → reduce resistance (if possible)
    if avg_reps < target_reps[0] and last_band_index > 0:
        #return band_levels[last_band_index - 1]
        return {"weight": band_levels[last_band_index - 1], "reps": avg_reps + 1}

    # Default: keep current band
    #return last_band
    return {"weight": last_band, "reps": avg_reps}


In [35]:
def test_band_algorithm_progressive():
    """
    Test the band_algorithm function with progressive workout data.
    """
    simulated_records = []
    band_levels = ["Light", "Medium", "Heavy", "Extra Heavy"]
    
    # Start with the lightest band and lower reps
    initial_record = {
        "phase": "Hypertrophy",
        "weight": band_levels[0],
        "reps": [9, 9, 9]
    }
    simulated_records.append(initial_record)

    # Run 10 test iterations
    for i in range(10):
        result = band_algorithm(simulated_records, "Hypertrophy")
        
        # Simulate user performance (slight variation in reps)
        user_performance = [max(result['reps'] - random.randint(0, 1), 8) for _ in range(3)]

        print(f"Day {i + 1}: Suggested Band: {result['weight']}, Suggested Reps: {result['reps']}, User Performance: {user_performance}")

        # Add the new record to the simulated records
        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": result['weight'],
            "reps": user_performance
        })

# Run the test
test_band_algorithm_progressive()


Day 1: Suggested Band: Light, Suggested Reps: 10, User Performance: [9, 9, 10]
Day 2: Suggested Band: Light, Suggested Reps: 10, User Performance: [9, 10, 9]
Day 3: Suggested Band: Light, Suggested Reps: 10, User Performance: [9, 9, 9]
Day 4: Suggested Band: Light, Suggested Reps: 10, User Performance: [9, 9, 9]
Day 5: Suggested Band: Light, Suggested Reps: 10, User Performance: [9, 10, 9]
Day 6: Suggested Band: Light, Suggested Reps: 10, User Performance: [9, 9, 10]
Day 7: Suggested Band: Light, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 8: Suggested Band: Light, Suggested Reps: 11, User Performance: [11, 10, 11]
Day 9: Suggested Band: Light, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 10: Suggested Band: Light, Suggested Reps: 12, User Performance: [11, 12, 12]


In [36]:
def test_band_algorithm_consistent():
    """
    Test the band_algorithm function where user performance always matches the suggested values.
    """
    simulated_records = []
    band_levels = ["Light", "Medium", "Heavy", "Extra Heavy"]
    
    # Start with the lightest band and lower reps
    initial_record = {
        "phase": "Hypertrophy",
        "weight": band_levels[0],
        "reps": [9, 9, 9]
    }
    simulated_records.append(initial_record)

    # Run 10 test iterations
    for i in range(10):
        result = band_algorithm(simulated_records, "Hypertrophy")

        # User performance matches the suggested reps
        user_performance = [result['reps']] * 3

        print(f"Day {i + 1}: Suggested Band: {result['weight']}, Suggested Reps: {result['reps']}, User Performance: {user_performance}")

        # Add the new record to the simulated records
        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": result['weight'],
            "reps": user_performance
        })

# Run the test
test_band_algorithm_consistent()


Day 1: Suggested Band: Light, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 2: Suggested Band: Light, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 3: Suggested Band: Light, Suggested Reps: 12, User Performance: [12, 12, 12]
Day 4: Suggested Band: Medium, Suggested Reps: 9, User Performance: [9, 9, 9]
Day 5: Suggested Band: Medium, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 6: Suggested Band: Medium, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 7: Suggested Band: Medium, Suggested Reps: 12, User Performance: [12, 12, 12]
Day 8: Suggested Band: Heavy, Suggested Reps: 9, User Performance: [9, 9, 9]
Day 9: Suggested Band: Heavy, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 10: Suggested Band: Heavy, Suggested Reps: 11, User Performance: [11, 11, 11]


In [37]:
def muscle_algorithm(similar_exercise_records, quantity):
    """
    Suggests an estimated weight for an exercise based on previous records of similar exercises.

    Parameters:
    - similar_exercise_records (list): User's past records for similar exercises.

    Returns:
    - dictionary: Suggested starting weight.
    """

    # Define a universal scaling factor (80% of similar exercise weight)
    SCALING_FACTOR = 0.8

    # Get the most recent record of the similar exercise
    latest_record = similar_exercise_records[-1]  
    last_weight = latest_record["weight"]  # Last used weight for similar exercise
    last_reps = latest_record["reps"]
    avg_reps = sum(last_reps) // len(last_reps)


    # Apply the universal scaling factor
    #TODO: NEED TO ROUND BASED ON THE FACT IF ITS DUMBBELL KETTLEBELL OR NOT
    estimated_weight = round_gym_weight(last_weight * SCALING_FACTOR, quantity > 1)

    #return estimated_weight
    return {"weight": estimated_weight, "reps": avg_reps}


In [38]:
def test_muscle_algorithm():
    """
    Test the muscle_algorithm function with simulated similar exercise records.
    """
    # Simulated similar exercise records
    similar_exercise_records = [
        {"phase": "Hypertrophy", "weight": 150, "reps": [10, 10, 10]},
        {"phase": "Hypertrophy", "weight": 160, "reps": [9, 9, 9]},
        {"phase": "Hypertrophy", "weight": 170, "reps": [8, 8, 8]},
        {"phase": "Hypertrophy", "weight": 165, "reps": [10, 10, 10]},
        {"phase": "Hypertrophy", "weight": 175, "reps": [9, 9, 9]}
    ]

    # Run the muscle algorithm
    result = muscle_algorithm(similar_exercise_records, 1)

    print(f"Suggested Weight: {result['weight']} lbs, Suggested Reps: {result['reps']}")

# Run the test
test_muscle_algorithm()


Suggested Weight: 140 lbs, Suggested Reps: 9


In [39]:
def test_weight_algorithm():
    """
    Test the weight_algorithm function with simulated workout data.
    """
    # Simulated workout records for Hypertrophy phase
    simulated_records = []
    weights = [110, 100, 110, 100, 110, 120, 110, 120, 130, 120]
    reps_list = [[9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12], [11, 11, 11], 
                 [9, 9, 9], [12, 12, 12], [10, 10, 10], [9, 9, 9], [11, 11, 11]]

    for weight, reps in zip(weights, reps_list):
        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": weight,
            "reps": reps
        })

    # Run 10 test iterations
    for i in range(10):
        result = weight_algorithm(simulated_records, "Hypertrophy")
        
        # Simulate the user's actual performance
        user_performance = [result['reps']] * 3  # Assume the user performs exactly as suggested
        
        print(f"Day {i + 1}: Suggested Weight: {result['weight']} lbs, Suggested Reps: {result['reps']}, User Performance: {user_performance}")

        # Add the new record to the simulated records
        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": result['weight'],
            "reps": user_performance
        })

# Run the test
test_weight_algorithm()


Day 1: Suggested Weight: 130 lbs, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 2: Suggested Weight: 120 lbs, Suggested Reps: 12, User Performance: [12, 12, 12]
Day 3: Suggested Weight: 130 lbs, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 4: Suggested Weight: 140 lbs, Suggested Reps: 9, User Performance: [9, 9, 9]
Day 5: Suggested Weight: 140 lbs, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 6: Suggested Weight: 130 lbs, Suggested Reps: 12, User Performance: [12, 12, 12]
Day 7: Suggested Weight: 140 lbs, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 8: Suggested Weight: 150 lbs, Suggested Reps: 9, User Performance: [9, 9, 9]
Day 9: Suggested Weight: 150 lbs, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 10: Suggested Weight: 140 lbs, Suggested Reps: 12, User Performance: [12, 12, 12]


In [40]:
def test_weight_algorithm_with_varied_performance():
    simulated_records = []
    weights = [110, 100, 110, 100, 110, 120, 110, 120, 130, 120, 130, 120, 130, 140, 130, 140, 150, 140, 150, 140]
    reps_list = [[9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12], [11, 11, 11],
                 [9, 9, 9], [12, 12, 12], [10, 10, 10], [9, 9, 9], [11, 11, 11],
                 [10, 10, 10], [12, 12, 12], [11, 11, 11], [9, 9, 9], [12, 12, 12],
                 [10, 10, 10], [9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12]]

    for weight, reps in zip(weights, reps_list):
        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": weight,
            "reps": reps
        })

    for i in range(10):
        result = weight_algorithm(simulated_records, "Hypertrophy")
        next_weight, next_reps = result["weight"], result["reps"]

        user_performance = [max(next_reps - random.randint(0, 1), 8) for _ in range(3)]

        print(f"Day {i + 1}: {next_weight} lbs, Suggested Reps: {next_reps}, User Performance: {user_performance}")

        simulated_records.append({
            "phase": "Hypertrophy",
            "weight": next_weight,
            "reps": user_performance
        })

# Run the test
test_weight_algorithm_with_varied_performance()


Day 1: 150 lbs, Suggested Reps: 11, User Performance: [10, 10, 11]
Day 2: 150 lbs, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 3: 160 lbs, Suggested Reps: 9, User Performance: [8, 9, 8]
Day 4: 160 lbs, Suggested Reps: 9, User Performance: [8, 9, 8]
Day 5: 160 lbs, Suggested Reps: 9, User Performance: [8, 9, 8]
Day 6: 160 lbs, Suggested Reps: 9, User Performance: [8, 8, 9]
Day 7: 160 lbs, Suggested Reps: 9, User Performance: [9, 8, 9]
Day 8: 160 lbs, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 9: 150 lbs, Suggested Reps: 12, User Performance: [11, 12, 12]
Day 10: 160 lbs, Suggested Reps: 11, User Performance: [11, 10, 10]


In [41]:
# Simulating 20 workouts with varied progression for Hypertrophy phase
simulated_records = []
weights = [110, 100, 110, 100, 110, 120, 110, 120, 130, 120, 130, 120, 130, 140, 130, 140, 150, 140, 150, 140]
reps_list = [[9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12], [11, 11, 11], 
             [9, 9, 9], [12, 12, 12], [10, 10, 10], [9, 9, 9], [11, 11, 11], 
             [10, 10, 10], [12, 12, 12], [11, 11, 11], [9, 9, 9], [12, 12, 12], 
             [10, 10, 10], [9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12]]  # Added variety

for weight, reps in zip(weights, reps_list):
    simulated_records.append({
        "phase": "Hypertrophy",
        "weight": weight,
        "reps": reps  # Reps as a list to account for variability across sets
    })

# Testing the function with simulated data
for i in range(10):
    results = weight_algorithm(simulated_records, "Hypertrophy")
    next_weight, next_reps = results['weight'], results['reps']
    
    # Simulating realistic user performance (slight variation in reps per set)
    user_performance = [max(next_reps - random.randint(0, 1), 8) for _ in range(3)]
    
    print(f"Day {i + 1}: {next_weight} lbs, Suggested Reps: {next_reps}, User Performance: {user_performance}")
    
    # Adding simulated performance to the records
    simulated_records.append({
        "phase": "Hypertrophy",
        "weight": next_weight,
        "reps": user_performance
    })


Day 1: 150 lbs, Suggested Reps: 11, User Performance: [10, 10, 11]
Day 2: 150 lbs, Suggested Reps: 11, User Performance: [11, 10, 10]
Day 3: 150 lbs, Suggested Reps: 11, User Performance: [10, 10, 11]
Day 4: 150 lbs, Suggested Reps: 11, User Performance: [10, 11, 10]
Day 5: 150 lbs, Suggested Reps: 11, User Performance: [10, 11, 10]
Day 6: 150 lbs, Suggested Reps: 11, User Performance: [11, 11, 10]
Day 7: 160 lbs, Suggested Reps: 9, User Performance: [8, 8, 8]
Day 8: 160 lbs, Suggested Reps: 9, User Performance: [8, 8, 8]
Day 9: 160 lbs, Suggested Reps: 9, User Performance: [9, 9, 8]
Day 10: 150 lbs, Suggested Reps: 11, User Performance: [10, 10, 10]


In [42]:
# Simulating 20 workouts with consistent progression for Hypertrophy phase
simulated_records = []
weights = [110, 100, 110, 100, 110, 120, 110, 120, 130, 120, 130, 120, 130, 140, 130, 140, 150, 140, 150, 140]
reps_list = [[9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12], [11, 11, 11], 
             [9, 9, 9], [12, 12, 12], [10, 10, 10], [9, 9, 9], [11, 11, 11], 
             [10, 10, 10], [12, 12, 12], [11, 11, 11], [9, 9, 9], [12, 12, 12], 
             [10, 10, 10], [9, 9, 9], [11, 11, 11], [10, 10, 10], [12, 12, 12]]

# Initializing the records with predefined data
for weight, reps in zip(weights, reps_list):
    simulated_records.append({
        "phase": "Hypertrophy",
        "weight": weight,
        "reps": reps  # Reps as a list to ensure uniformity across sets
    })

# Testing the function with simulated data
for i in range(10):
    results = weight_algorithm(simulated_records, "Hypertrophy")
    
    # Simulating user performance: always performing the suggested reps
    user_performance = [results['reps']] * 3
    
    print(f"Day {i + 1}: {results['weight']} lbs, Suggested Reps: {results['reps']}, User Performance: {user_performance}")
    
    # Adding the consistent user performance to the records
    simulated_records.append({
        "phase": "Hypertrophy",
        "weight": results['weight'],
        "reps": user_performance
    })


Day 1: 150 lbs, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 2: 160 lbs, Suggested Reps: 9, User Performance: [9, 9, 9]
Day 3: 160 lbs, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 4: 150 lbs, Suggested Reps: 12, User Performance: [12, 12, 12]
Day 5: 160 lbs, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 6: 170 lbs, Suggested Reps: 9, User Performance: [9, 9, 9]
Day 7: 170 lbs, Suggested Reps: 10, User Performance: [10, 10, 10]
Day 8: 160 lbs, Suggested Reps: 12, User Performance: [12, 12, 12]
Day 9: 170 lbs, Suggested Reps: 11, User Performance: [11, 11, 11]
Day 10: 180 lbs, Suggested Reps: 9, User Performance: [9, 9, 9]


In [43]:
# experience_multiplier = {
#     "Newcomer": 1,   # Lightest, cautious approach
#     "Beginner": 1.25,    # Standard weight suggestion
#     "Intermediate": 1.50, # More load as they progress
#     "Advanced": 2.     # Higher challenge for experienced lifters
# }

In [44]:
experience_multiplier = {
    '1': 1,   # Lightest, cautious approach
    '2': 1.25,    # Standard weight suggestion
    '3': 1.50, # More load as they progress
    '4': 2.0     # Higher challenge for experienced lifters
}

In [45]:
#NOTE: This is the old version of this function. Can be skipped

# ###############################################################################################

# def find_similar_exercise(exercise_name, records, user_id, filtered_dataset):
#     """
#     Finds a similar exercise using the "Variations" column from the dataset.

#     Parameters:
#     - exercise_name (str): The exercise needing an estimate.
#     - records (dict): User's exercise records.
#     - user_id (str): User's unique ID.
#     - filtered_dataset (DataFrame): The filtered exercise dataset containing variations.

#     Returns:
#     - str or None: A similar exercise name if found.
#     """
#     # Find the row corresponding to the given exercise
#     exercise_row = filtered_dataset[filtered_dataset["Exercise"] == exercise_name]

#     if exercise_row.empty:
#         return None  # No match found in dataset

#     # Get the list of variations for the exercise
#     variations = exercise_row.iloc[0]["Variations"]

#     # Ensure variations exist and are stored as a list
#     if isinstance(variations, list):
#         for variation in variations:
#             if user_id in records and variation in records[user_id]:
#                 return variation  # Return the first available variation

#     return None  # No suitable variation found

################################################################################################

In [46]:
# #AGAIN, SHOULD WE JUST PASS THE USER'S RECORD TO NARROW DOWN THE FLOW OF INFORMATION.
# #CREATE A FUNCTION THAT RETRIEVES THESE RECORDS

def find_similar_exercise(exercise_name, records, user_id, filtered_dataset):
    """
    Finds a similar exercise using the "Variations" column from the dataset.

    Parameters:
    - exercise_name (str): The exercise needing an estimate.
    - records (dict): User's exercise records.
    - user_id (str): User's unique ID.
    - filtered_dataset (DataFrame): The filtered exercise dataset containing variations.

    Returns:
    - str or None: A similar exercise name if found.
    """
    # Find the row corresponding to the given exercise
    exercise_row = filtered_dataset[filtered_dataset["Exercise"] == exercise_name]

    if exercise_row.empty:
        return None  # No match found in dataset

    # Get the list of variations for the exercise
    variations = exercise_row.iloc[0].get("Variations", [])

    # Ensure variations exist and are stored as a list
    if isinstance(variations, list):
        for variation in variations:
            if user_id in records and variation in records[user_id]["by_exercise"]:
                return variation  # Return the first available variation
                #NOTE TODO: Could, instead of returning the name of the variation, return the records for the exercise right away

    return None  # No suitable variation found


In [47]:
def find_specific_equipment(exercise_equipment):
    """
    Infers the specific equipment type for gym equipment exercises.

    Parameters:
    - exercise_equipment (list of lists): Each sublist represents a possible combination of equipment to perform the exercise.

    Returns:
    - Tuple(str, int): The specific equipment type and quantity needed.
    """
    equipment_type_map = {
        "1 Dumbbell":         ("Dumbbells", 1),
        "2 Dumbbell":         ("Dumbbells", 2),
        "1 Kettlebell":       ("Kettlebells", 1),
        "2 Kettlebell":       ("Kettlebells", 2),
        "Fixed weight bar":   ("Fixed weight bar", 1),
        "Mini loop band":     ("Mini loop band", 1),
        "1 Loop band":        ("Loop band", 1),
        "2 Loop band":        ("Loop band", 2),
        "Handle band":        ("Handle band", 1)
    }

    for combo in exercise_equipment:  # combo is a list like ['1 Loop band', '1 Single grip handle']
        #print("printing combo", combo)
        for item in combo:
            #print("printing item",item)
            if item in equipment_type_map:
                return equipment_type_map[item]

    return None  # If no known equipment is found

In [48]:
def find_closest_available_weight(suggested_weight, equipment_type, user_available_weights, required_quantity=1):
    """
    Finds the closest available weight the user owns for the given equipment type,
    considering the required quantity. In case of a tie, returns the lower weight.

    Parameters:
    - suggested_weight (float): The desired weight to suggest.
    - equipment_type (str): The type of equipment (e.g., 'Dumbbells', 'Kettlebells', 'Fixed weight bar').
    - user_available_weights (dict): User's available weights with quantities.
    - required_quantity (int): How many of the weight are needed (default is 1).

    Returns:
    - float or None: Closest available weight (rounded down on tie), or None if no valid options exist.
    """

    if required_quantity == 2:
        suggested_weight = suggested_weight / 2


    available = user_available_weights.get(equipment_type, {})

    # Filter only weights with enough quantity
    valid_weights = [w for w, qty in available.items() if qty >= required_quantity]

    if not valid_weights:
        return None

    # Sort by distance from target weight, with tiebreaker: lower weight wins
    valid_weights.sort(key=lambda w: (abs(w - suggested_weight), w))

    if required_quantity == 2:
        return valid_weights[0] * 2
    else:
        return valid_weights[0]


In [49]:
def find_closest_available_resistance(suggested_level, equipment_type, user_available_weights, required_quantity=1):
    """
    Finds the closest available resistance level from user's inventory.

    Parameters:
    - suggested_level (str): The desired resistance band level (e.g., 'Medium').
    - equipment_type (str): The band type (e.g., 'Mini loop band').
    - user_available_weights (dict): Dictionary of user's available bands and counts.
    - required_quantity (int): Number of bands needed (default is 1).

    Returns:
    - str or None: Closest resistance level the user has, or None if none available.
    """

    RESISTANCE_BAND_LEVELS = ["Extra Light", "Light", "Medium", "Heavy", "Extra Heavy"]

    if equipment_type not in user_available_weights:
        return None

    available_bands = user_available_weights[equipment_type]
    
    if suggested_level not in RESISTANCE_BAND_LEVELS:
        return None

    target_index = RESISTANCE_BAND_LEVELS.index(suggested_level)

    # Search outward from the suggested level
    for offset in range(len(RESISTANCE_BAND_LEVELS)):
        # Check lighter
        lower_index = target_index - offset
        if lower_index >= 0:
            level = RESISTANCE_BAND_LEVELS[lower_index]
            if available_bands.get(level, 0) >= required_quantity:
                return level
        
        # Check heavier
        upper_index = target_index + offset
        if upper_index < len(RESISTANCE_BAND_LEVELS):
            level = RESISTANCE_BAND_LEVELS[upper_index]
            if available_bands.get(level, 0) >= required_quantity:
                return level

    return None  # No valid band found

In [50]:
#AGAIN, SHOULD WE JUST PASS THE USER'S RECORD TO NARROW DOWN THE FLOW OF INFORMATION.

def determine_weight(row, user_id, user_level, records, filtered_dataset, training_phase, user_available_weights, user_equipment):
    """
    Determines weight based on user history or estimates using experience multipliers.

    Parameters:
    - row (Series): Row of the DataFrame containing exercise details.
    - user_id (str): Unique user ID.
    - user_level (str): User's experience level.
    - records (dict): Dictionary storing past lift records.
    - filtered_dataset (DataFrame): exercise dataset.
    - training_phase (str): User's current training phase.
    - user_available_weights (dictionary): Dictionary containing keys (i.e. "Dumbbells") and subkey:value pairs?

    Returns:
    - dict: {"weight": suggested_weight, "reps": suggested_reps}
    """

    exercise_name = row["Exercise"]
    min_weight = row["Lower bound (lbs/resistance/time)"]
    exercise_type = infer_equipment_type(min_weight)
    exercise_equipment = row["Equipment"]
    #print("printing equipment", row["Equipment"])

    #MIGHT NEED TO CHANGE THE WAY WE ACCESS RECORDS. HOW ARE WE GONNA STORE THEM?

    # If user has past records, apply progressive overload
    # if user_id in records and exercise_name in records[user_id]["by_exercise"]:

    #NOTE: THE FOLLOWING LINE RETURNS BOOLEAN
    if records.get(user_id) and exercise_name in records[user_id]["by_exercise"]:
        # Filter records specific to the current training phase (done in weight_algorithm, etc)
        exercise_records = records[user_id]["by_exercise"][exercise_name]

        if exercise_type == "Gym Equipment":
            #NOTE: Weight_algorithm calls Brzycki and Epley formulas, which expect TOTAL values
            #Exercise_records should be saved as total weights.
            suggested_results = weight_algorithm(exercise_records, training_phase)
            print("Calling weight algorithm with records")
        elif exercise_type == "Resistance Band":
            suggested_results = band_algorithm(exercise_records, training_phase)
            print("Calling band algorithm with records")
        elif exercise_type == "Bodyweight":
            print("Calling bodyweight algorithm with records")
            return bodyweight_algorithm(exercise_records, training_phase)
        elif exercise_type == "Timed Exercise":
            print("Calling timed algorithm with records")
            return timed_algorithm(exercise_records)

        suggested_weight, suggested_reps = suggested_results["weight"], suggested_results["reps"]
        equipment_info = find_specific_equipment(exercise_equipment)
        print("printing equipment info: ", equipment_info)

        if equipment_info:
            #i.e. Dumbbell, 2
            print("inside equipment_info statement")
            equipment_type, quantity_needed = equipment_info

            skip_validation = (
                equipment_type == "Fixed weight bar" and 
                any(e in user_equipment for e in ["Olympic barbell", "EZ curl bar"])
            )

            if not skip_validation: 
                single_weight = suggested_weight
                if equipment_type in ["Dumbbells", "Kettlebells"] and quantity_needed == 2:
                    single_weight = suggested_weight / 2
                    user_has_weight = user_available_weights.get(equipment_type, {}).get(single_weight, 0)
                else:
                    user_has_weight = user_available_weights.get(equipment_type, {}).get(suggested_weight, 0)

                if user_has_weight < quantity_needed:
                    print(f"⚠️ Adjusting weight: {single_weight} not available!")
                    if exercise_type == "Gym Equipment":
                        suggested_weight = find_closest_available_weight(suggested_weight, equipment_type, user_available_weights, quantity_needed)
                    elif exercise_type == "Resistance Band":
                        suggested_weight = find_closest_available_resistance(suggested_weight, equipment_type, user_available_weights, quantity_needed)

        return {"weight": suggested_weight, "reps": suggested_reps}


    # No previous records for this exercise? Check similar exercises
    if exercise_type == "Gym Equipment":
        equipment_info = find_specific_equipment(exercise_equipment)
        quantity_needed = 1

        #################################################################################################
        #equipment_type = None 
        #################################################################################################


        if equipment_info:
                #i.e. Dumbbell, 2
                print("Special case: equipment info")
                print(equipment_info)
                equipment_type, quantity_needed = equipment_info
        
        similar_exercise = find_similar_exercise(exercise_name, records, user_id, filtered_dataset)
        if similar_exercise:
            similar_records = records[user_id]["by_exercise"][similar_exercise]
            print("About to call muscle_algorithm()...")
                
            suggested_results = muscle_algorithm(similar_records, quantity_needed) 
            suggested_weight = suggested_results["weight"]
            suggested_reps = suggested_results["reps"]

        else:
            # No similar exercise for Gym equipment? Estimate starting weight
            print("Estimating starting weight based on experience multiplier...")
            suggested_weight = min_weight * quantity_needed * experience_multiplier[user_level]
            suggested_weight = round_gym_weight(suggested_weight, quantity_needed > 1)
            suggested_reps = 10

         
        if equipment_info:
        #print("I am at 2 skip validation")
            skip_validation = (
                equipment_type == "Fixed weight bar" and 
                any(e in user_equipment for e in ["Olympic barbell", "EZ curl bar"])
            )
            
            if not skip_validation: 
                single_weight = suggested_weight
                if equipment_type in ["Dumbbells", "Kettlebells"] and quantity_needed == 2:
                    single_weight = suggested_weight / 2
                    user_has_weight = user_available_weights.get(equipment_type, {}).get(single_weight, 0)
                else:
                    user_has_weight = user_available_weights.get(equipment_type, {}).get(suggested_weight, 0)

                if user_has_weight < quantity_needed:
                    print(f"⚠️ Adjusting weight 2: {single_weight} lbs not available!")
                    suggested_weight = find_closest_available_weight(suggested_weight, equipment_type, user_available_weights, quantity_needed)

        return {"weight": suggested_weight, "reps": suggested_reps}
    
    if exercise_type == "Resistance Band":
        suggested_weight = min_weight
        suggested_reps = 10

        equipment_info = find_specific_equipment(exercise_equipment)
        if equipment_info:
            #i.e. 2 Loop Band
            equipment_type, quantity_needed = equipment_info

            user_has_weight = user_available_weights.get(equipment_type, {}).get(suggested_weight, 0)
                
            if user_has_weight < quantity_needed:
                print(f"⚠️ Adjusting resistance: {suggested_weight} not available!")
                suggested_weight = find_closest_available_resistance(suggested_weight, equipment_type, user_available_weights, quantity_needed)

        return {"weight": suggested_weight, "reps": suggested_reps}
    
    if exercise_type == "Timed Exercise":
        print("No record timed exercise...")
        return {"weight": None, "reps": None, "time": min_weight}
    
    #return min_weight, keep unchanged for bodyweight
    #print("no record bodyweight exercise")
    return {"weight": min_weight, "reps": 10, "time": None}

# ChatGPT improved function

In [51]:
# Updated determine_weight function with refactoring

def new_determine_weight(row, user_id, user_level, records, filtered_dataset, training_phase, user_available_weights, user_equipment):
    exercise_name = row["Exercise"]
    min_weight = row["Lower bound (lbs/resistance/time)"]
    exercise_type = infer_equipment_type(min_weight)
    exercise_equipment = row["Equipment"]

    def get_equipment_info():
        info = find_specific_equipment(exercise_equipment)
        return info if info else (None, 1)

    def check_and_adjust_weight(weight, equipment_type, quantity_needed):
        skip = equipment_type == "Fixed weight bar" and any(e in user_equipment for e in ["Olympic barbell", "EZ curl bar"])
        if skip:
            return weight

        if equipment_type in ["Dumbbells", "Kettlebells"] and quantity_needed == 2:
            single_weight = weight / 2
        else:
            single_weight = weight

        user_has = user_available_weights.get(equipment_type, {}).get(single_weight, 0)
        if user_has < quantity_needed:
            print(f"⚠️ Adjusting weight: {single_weight} not available!")
            if exercise_type == "Resistance Band":
                return find_closest_available_resistance(weight, equipment_type, user_available_weights, quantity_needed)
            return find_closest_available_weight(weight, equipment_type, user_available_weights, quantity_needed)
        return weight

    # If user has past records
    if records.get(user_id) and exercise_name in records[user_id].get("by_exercise", {}):
        exercise_records = records[user_id]["by_exercise"][exercise_name]

        if exercise_type == "Gym Equipment":
            print("Calling weight algorithm with records")
            suggested = weight_algorithm(exercise_records, training_phase)
        elif exercise_type == "Resistance Band":
            print("Calling band algorithm with records")
            suggested = band_algorithm(exercise_records, training_phase)
        elif exercise_type == "Bodyweight":
            print("Calling bodyweight algorithm with records")
            return bodyweight_algorithm(exercise_records, training_phase)
        elif exercise_type == "Timed Exercise":
            print("Calling timed algorithm with records")
            return timed_algorithm(exercise_records)

        suggested_weight, suggested_reps = suggested["weight"], suggested["reps"]
        equipment_type, quantity_needed = get_equipment_info()
        if equipment_type:
            suggested_weight = check_and_adjust_weight(suggested_weight, equipment_type, quantity_needed)
        return {"weight": suggested_weight, "reps": suggested_reps}

    # No previous records
    if exercise_type == "Gym Equipment":
        equipment_type, quantity_needed = get_equipment_info()

        similar = find_similar_exercise(exercise_name, records, user_id, filtered_dataset)
        if similar:
            similar_records = records[user_id]["by_exercise"][similar]
            print("About to call muscle_algorithm()...")
            suggested = muscle_algorithm(similar_records, quantity_needed)
        else:
            print("Estimating starting weight based on experience multiplier...")
            suggested_weight = min_weight * quantity_needed * experience_multiplier[user_level]
            suggested_weight = round_gym_weight(suggested_weight, quantity_needed > 1)
            suggested = {"weight": suggested_weight, "reps": 10}

        suggested_weight, suggested_reps = suggested["weight"], suggested["reps"]
        if equipment_type:
            suggested_weight = check_and_adjust_weight(suggested_weight, equipment_type, quantity_needed)
        return {"weight": suggested_weight, "reps": suggested_reps}

    if exercise_type == "Resistance Band":
        suggested_weight, suggested_reps = min_weight, 10
        equipment_type, quantity_needed = get_equipment_info()
        if equipment_type:
            suggested_weight = check_and_adjust_weight(suggested_weight, equipment_type, quantity_needed)
        return {"weight": suggested_weight, "reps": suggested_reps}

    if exercise_type == "Timed Exercise":
        print("No record timed exercise...")
        return {"weight": None, "reps": None, "time": min_weight}

    print("no record bodyweight exercise")
    return {"weight": min_weight, "reps": 10, "time": None}

In [52]:
def test_determine_weight_comprehensive_plus():
    import pandas as pd

    # Simulated user records
    user_records = {
        "user123": {
            "by_exercise": {
                "Barbell Bench Press": [
                    {"phase": "Hypertrophy", "weight": 135, "reps": [10, 10, 10]},
                    {"phase": "Hypertrophy", "weight": 145, "reps": [9, 9, 9]},
                ],
                "Push-Ups": [
                    {"phase": "Hypertrophy", "weight": None, "reps": [9, 9, 9]},
                    {"phase": "Hypertrophy", "weight": None, "reps": [11, 11, 11]},
                ],
                "Plank": [
                    {"phase": "Endurance", "weight": None, "reps": None, "time": [60, 55, 50]},
                    {"phase": "Endurance", "weight": None, "reps": None, "time": [65, 60, 65]},
                ],
                "Banded Rows": [
                    {"phase": "Hypertrophy", "weight": "Medium", "reps": [10, 10, 10]},
                    {"phase": "Hypertrophy", "weight": "Heavy", "reps": [9, 9, 9]},
                ],
                "Front squat": [
                    {"phase": "Hypertrophy", "weight": 50, "reps": [10, 10, 10]},
                    {"phase": "Hypertrophy", "weight": 60, "reps": [8, 8, 8]},
                ]
            }
        }
    }

    # Simulated filtered dataset as DataFrame
    filtered_dataset = pd.DataFrame([
        {"Exercise": "Barbell Bench Press", "Lower bound (lbs/resistance/time)": 45,  "Variations": ["Incline Dumbbell Press"],
         "Equipment": [["Bench", "Olympic barbell"]]},

        {"Exercise": "Push-Ups", "Lower bound (lbs/resistance/time)": 0,
         "Equipment": [["None"]]},

        {"Exercise": "Plank", "Lower bound (lbs/resistance/time)": "60 seconds",
         "Equipment": [["None"]]},

        {"Exercise": "Banded Rows", "Lower bound (lbs/resistance/time)": "Light",
         "Equipment": [["2 Loop band", "2 Single grip handle"]]},

        {"Exercise": "Incline Dumbbell Press", "Lower bound (lbs/resistance/time)": 40, "Variations": ["Barbell Bench Press"],
         "Equipment": [["Incline Bench", "2 Dumbbell"]]},

        {"Exercise": "Front squat", "Lower bound (lbs/resistance/time)": 50,
         "Equipment": [["Fixed weight bar"]]},

        
        {"Exercise": "Deficit push up", "Lower bound (lbs/resistance/time)": 0,
         "Equipment": [["None"]]},

        {"Exercise": "Kneeling side plank", "Lower bound (lbs/resistance/time)": "60 seconds",
         "Equipment": [["None"]]}
    ])

    # Available weights and equipment
    user_available_weights = {
        "Dumbbells": {40: 2, 50: 2, 60:1},
        "Kettlebells": {50: 2},
        "Fixed weight bar": {50: 1},
        "Mini loop band": {"Light": 1, "Medium": 1, "Heavy": 1},
        "Loop band": {"Extra Light":2, "Light":2, "Medium":2}
    }

    user_equipment = [
        "Olympic barbell", "Bench", "2 Dumbbell", "2 Loop band", "2 Single grip handle", "Fixed weight bar"
    ]

    # Test exercises
    exercises_to_test = filtered_dataset.to_dict('records')
    user_level = '2'  # Intermediate
    training_phase = "Hypertrophy"

    print("🔍 Testing determine_weight() for various exercise types:\n")

    for exercise in exercises_to_test:
        result = determine_weight(
            row=exercise,
            user_id="user123",
            user_level=user_level,
            records=user_records,
            filtered_dataset=filtered_dataset,
            training_phase=training_phase,
            user_available_weights=user_available_weights,
            user_equipment=user_equipment
        )
        print(f"➡️ Exercise: {exercise['Exercise']}")
        if result.get("time") is not None:
            print(f"   Suggested Duration: {result['time']}")
        else:
            print(f"   Suggested Weight: {result['weight']}, Reps: {result['reps']}")
        print("—" * 50)

# Run the test
test_determine_weight_comprehensive_plus()

🔍 Testing determine_weight() for various exercise types:

Calling weight algorithm with records
printing equipment info:  None
➡️ Exercise: Barbell Bench Press
   Suggested Weight: 135, Reps: 11
——————————————————————————————————————————————————
Calling bodyweight algorithm with records
➡️ Exercise: Push-Ups
   Suggested Weight: None, Reps: 12
——————————————————————————————————————————————————
Calling timed algorithm with records
➡️ Exercise: Plank
   Suggested Duration: 68
——————————————————————————————————————————————————
Calling band algorithm with records
printing equipment info:  ('Loop band', 2)
inside equipment_info statement
⚠️ Adjusting weight: Heavy not available!
➡️ Exercise: Banded Rows
   Suggested Weight: Medium, Reps: 10
——————————————————————————————————————————————————
Special case: equipment info
('Dumbbells', 2)
About to call muscle_algorithm()...
⚠️ Adjusting weight 2: 60.0 lbs not available!
➡️ Exercise: Incline Dumbbell Press
   Suggested Weight: 100, Reps: 9
————

In [53]:
def test_determine_weight_comprehensive_plus_chatgpt():
    import pandas as pd

    # Simulated user records
    user_records = {
        "user123": {
            "by_exercise": {
                "Barbell Bench Press": [
                    {"phase": "Hypertrophy", "weight": 135, "reps": [10, 10, 10]},
                    {"phase": "Hypertrophy", "weight": 145, "reps": [9, 9, 9]},
                ],
                "Push-Ups": [
                    {"phase": "Hypertrophy", "weight": None, "reps": [9, 9, 9]},
                    {"phase": "Hypertrophy", "weight": None, "reps": [11, 11, 11]},
                ],
                "Plank": [
                    {"phase": "Endurance", "weight": None, "reps": None, "time": [60, 55, 50]},
                    {"phase": "Endurance", "weight": None, "reps": None, "time": [65, 60, 65]},
                ],
                "Banded Rows": [
                    {"phase": "Hypertrophy", "weight": "Medium", "reps": [10, 10, 10]},
                    {"phase": "Hypertrophy", "weight": "Heavy", "reps": [9, 9, 9]},
                ],
                "Front squat": [
                    {"phase": "Hypertrophy", "weight": 50, "reps": [10, 10, 10]},
                    {"phase": "Hypertrophy", "weight": 60, "reps": [8, 8, 8]},
                ]
            }
        }
    }

    # Simulated filtered dataset as DataFrame
    filtered_dataset = pd.DataFrame([
        {"Exercise": "Barbell Bench Press", "Lower bound (lbs/resistance/time)": 45,  "Variations": ["Incline Dumbbell Press"],
         "Equipment": [["Bench", "Olympic barbell"]]},

        {"Exercise": "Push-Ups", "Lower bound (lbs/resistance/time)": 0,
         "Equipment": [["None"]]},

        {"Exercise": "Plank", "Lower bound (lbs/resistance/time)": "60 seconds",
         "Equipment": [["None"]]},

        {"Exercise": "Banded Rows", "Lower bound (lbs/resistance/time)": "Light",
         "Equipment": [["2 Loop band", "2 Single grip handle"]]},

        {"Exercise": "Incline Dumbbell Press", "Lower bound (lbs/resistance/time)": 40, "Variations": ["Barbell Bench Press"],
         "Equipment": [["Incline Bench", "2 Dumbbell"]]},

        {"Exercise": "Front squat", "Lower bound (lbs/resistance/time)": 50,
         "Equipment": [["Fixed weight bar"]]},

        
        {"Exercise": "Deficit push up", "Lower bound (lbs/resistance/time)": 0,
         "Equipment": [["None"]]},

        {"Exercise": "Kneeling side plank", "Lower bound (lbs/resistance/time)": "60 seconds",
         "Equipment": [["None"]]}
    ])

    # Available weights and equipment
    user_available_weights = {
        "Dumbbells": {40: 2, 50: 2, 60:1},
        "Kettlebells": {50: 2},
        "Fixed weight bar": {50: 1},
        "Mini loop band": {"Light": 1, "Medium": 1, "Heavy": 1},
        "Loop band": {"Extra Light":2, "Light":2, "Medium":2}
    }

    user_equipment = [
        "Olympic barbell", "Bench", "2 Dumbbell", "2 Loop band", "2 Single grip handle", "Fixed weight bar"
    ]

    # Test exercises
    exercises_to_test = filtered_dataset.to_dict('records')
    user_level = '2'  # Intermediate
    training_phase = "Hypertrophy"

    print("🔍 Testing determine_weight() for various exercise types:\n")

    for exercise in exercises_to_test:
        result = new_determine_weight(
            row=exercise,
            user_id="user123",
            user_level=user_level,
            records=user_records,
            filtered_dataset=filtered_dataset,
            training_phase=training_phase,
            user_available_weights=user_available_weights,
            user_equipment=user_equipment
        )
        print(f"➡️ Exercise: {exercise['Exercise']}")
        if result.get("time") is not None:
            print(f"   Suggested Duration: {result['time']}")
        else:
            print(f"   Suggested Weight: {result['weight']}, Reps: {result['reps']}")
        print("—" * 50)

# Run the test
test_determine_weight_comprehensive_plus_chatgpt()



🔍 Testing determine_weight() for various exercise types:

Calling weight algorithm with records
➡️ Exercise: Barbell Bench Press
   Suggested Weight: 135, Reps: 11
——————————————————————————————————————————————————
Calling bodyweight algorithm with records
➡️ Exercise: Push-Ups
   Suggested Weight: None, Reps: 12
——————————————————————————————————————————————————
Calling timed algorithm with records
➡️ Exercise: Plank
   Suggested Duration: 68
——————————————————————————————————————————————————
Calling band algorithm with records
⚠️ Adjusting weight: Heavy not available!
➡️ Exercise: Banded Rows
   Suggested Weight: Medium, Reps: 10
——————————————————————————————————————————————————
About to call muscle_algorithm()...
⚠️ Adjusting weight: 60.0 not available!
➡️ Exercise: Incline Dumbbell Press
   Suggested Weight: 100, Reps: 9
——————————————————————————————————————————————————
Calling weight algorithm with records
➡️ Exercise: Front squat
   Suggested Weight: 60, Reps: 9
———————————————

# COMPRENHENSIVE TEST

In [64]:
path = 'dataset_3.json'
df = pd.read_json(path)

############################################################################################
# CONSTANT

age = 28              #Based on birthday?
avg_time_per_set = 1  # Average time per set in minutes
sets_per_exercise = 4  # Number of sets per exercise
current_day = 0        # Increases by one after every completed workout
user_records = { 
        
    "user123": {
         "by_exercise": {
              
         }   
    }
}

# USER SPECIFICATIONS FROM THE ONBOARDING OR SETTINGS (CAN BE CHANGED)

days_of_week = [1,2,3,4,5,6,7]
workout_frequency = 6
time_per_workout = 120
level = '4'
user_goals = ['Bodybuilding', 'Build muscles', 'Aesthetics', 'Get stronger']
pp = []
equipment = gym_equipment["Fully equipped gym"]["equipment"]
user_available_weights = gym_equipment["Fully equipped gym"]["available_weights"]


priority_muscles = ["Shoulders"]

#TODO: MODIFY THESE BEFORE TESTING. 
user_favorites = {"Barbell Bench Press"}
suggest_less = {}
dont_show_again = {"Incline Dumbbell Press"}




############################################################################################

#Get the split (push-pull-legs)
Jose_split = recommend_split(days_of_week,workout_frequency,time_per_workout,level,user_goals)
print(Jose_split)

#Get all(in this case one) modalities
modality = goal_to_modality_further_simplified.get(user_goals[0])    #!!!!!!!!!!!!!!!!!!!
print(modality)

training_phase, muscle_group_index = get_training_phase_and_group_for_day(user_goals, Jose_split, current_day)


rest_time_per_set = get_reps_and_rest_time(training_phase)["rest_time"]

muscle_group = split_dictionary_complex[Jose_split]["groups"][muscle_group_index]
range_focus_distribution = split_dictionary_complex[Jose_split]["focus_distribution_ranges"][muscle_group_index]

#filter the main data
primary_filter = filter_data(df,level,equipment,modality,age,pp)
#print(f"Total rows after primary filtering: {primary_filter.shape[0]}")

#FILTER BASED ON THE MUSCLE GROUP OF THE DAY, THIS CASE, PUSH DAY MUSCLE GROUP
secondary_filter = filter_muscles(primary_filter, muscle_group)
#print(f"Total rows after secondary filtering: {secondary_filter.shape[0]}")


#DISTRIBUTION FUNCTION #2
randomized_focus_distribution = generate_biased_distribution(range_focus_distribution, priority_muscles, muscle_group)
#print("distribution: " ,randomized_focus_distribution)
#print("✅ Sum of randomized distribution + bias:", sum(randomized_focus_distribution))

total_exercises = calculate_total_exercises(time_per_workout,avg_time_per_set, sets_per_exercise,rest_time_per_set)


exercises_with_focus = calculate_exercises_per_muscle(total_exercises, muscle_group,randomized_focus_distribution)
print(f"✅ Exercises per muscle: {exercises_with_focus}\n")


exercises = select_exercises(secondary_filter,exercises_with_focus)
for muscle_group, df in exercises.items():
        print(f"\nMuscle group: {muscle_group}\n")
        for _, exercise in df.iterrows():
            result = determine_weight(
                row=exercise,
                user_id="user123",
                user_level=level,
                records=user_records,
                filtered_dataset=secondary_filter,
                training_phase=training_phase,
                user_available_weights=user_available_weights,
                user_equipment=equipment
                
            )
            print(f"Exercise: {exercise['Exercise']}, Suggested: {result}")


Push-Pull-Legs
['H']
✅ Exercises per muscle: {'Chest': 3, 'Shoulders': 3, 'Triceps': 1}


Muscle group: Chest

Special case: equipment info
('Fixed weight bar', 1)
Estimating starting weight based on experience multiplier...
Exercise: Close-grip barbell floor press, Suggested: {'weight': 30, 'reps': 10}
Estimating starting weight based on experience multiplier...
Exercise: Incline chest press machine, Suggested: {'weight': 90, 'reps': 10}
Estimating starting weight based on experience multiplier...
Exercise: Close-grip bench press, Suggested: {'weight': 90, 'reps': 10}

Muscle group: Shoulders

Estimating starting weight based on experience multiplier...
Exercise: Reverse cable crossover, Suggested: {'weight': 20, 'reps': 10}
Special case: equipment info
('Dumbbells', 2)
Estimating starting weight based on experience multiplier...
Exercise: Lean-away dumbbell lateral raise, Suggested: {'weight': 10, 'reps': 10}
Estimating starting weight based on experience multiplier...
Exercise: Seat